## Solving Tic-Tac-Toe using TD(λ)
In this assignment you will build an RL agent capable of playing Tic-Tac-Toe using TD(λ) algorithm and the environment simulated by you in first week.

First of all copy the environment simulated by you from the first week below.
- Note that you should also return the state of the board each time you call act function, ideally the state should be stored in a numpy array for faster implementation
- The only input the function can take is via its own arguments and not input() function.

The ideal TicTacToe environment:
- Will take N, the size of board as an argument in its constructor.
- Will have act function taking a single argument representing the action taken (preferably int type) and return the state of board (preferably numpy array), reward signal (float) and bool value "done" which is true if the game is over else false.
- Will have reset function which resets the board and starts a new game.
- Will give reward signal as 1 if 'X' won and -1 if 'O' won (or vice-versa) and 0 if its a draw.
- Will take alternate calls of act function as the moves of one player.

For example:
```html
env.reset()
Returns ==> (array([[0., 0., 0.],[0., 0., 0.],[0., 0., 0.]]), 0, False)
                | | | |
        Board:  | | | |
                | | | |

env.act(4)
Returns ==> (array([[0., 0., 0.],[0., 1.0, 0.],[0., 0., 0.]]), 0, False)
                | | | |
        Board:  | |X| |
                | | | |

env.act(0)
Returns ==> (array([[-1.0, 0., 0.],[0., 1.0, 0.],[0., 0., 0.]]), 0, False)
                |O| | |
        Board:  | |X| |
                | | | |

env.act(7)
Returns ==> (array([[-1.0, 0., 0.],[0., 1.0, 0.],[0., 1.0, 0.]]), 0, False)
                |O| | |
        Board:  | |X| |
                | |X| |

env.act(6)
Returns ==> (array([[-1.0, 0., 0.],[0., 1.0, 0.],[-1.0, 1.0, 0.]]), 0, False)
                |O| | |
        Board:  | |X| |
                |O|X| |

env.act(2)
Returns ==> (array([[-1.0, 1.0, 0.],[0., 1.0, 0.],[-1.0, 1.0, 0.]]), 1, True)
                |O|X| |
        Board:  | |X| |
                |O|X| |


```
<hr>

Note : You can change your TicTacToe environment code before using it here


In [45]:
# Import any necessary libraries here
import numpy as np
import random

Your TicTacToe environment class comes here

In [46]:
class TicTacToe:
    def __init__(self, n : int = 3):
        self.n = n
        self.grid = np.full((n, n), 0)
        self.currMove = 1
        self.moveNum = 0
        self.xMove = None
        self.yMove = None


    # Given below is the preferable structure of act function
    def act(self, action : int) -> tuple:     # Returns tuple of types (np.ndarray, int, bool)
        self.action = action
        self.xMove = action // self.n
        self.yMove = action % self.n
        maxMove = self.n ** 2
        if action < 0 or action >= maxMove or self.grid[self.xMove][self.yMove] != 0:
            print("Wrong Move, try again!")
            return self.grid, 0, False
        self.moveNum += 1
        self.grid[self.xMove][self.yMove] = self.currMove
        if self._checkWin() == True:
            if self.currMove == 1:
                return self.grid, 1, True
            return self.grid, -1, True
        if self.moveNum == maxMove:
            return self.grid, 0, True
        self._changeMove()
        return self.grid, 0, False

    def reset(self):
        self.grid = np.full((self.n, self.n), 0)
        self.currMove = 1
        return self.grid, 0, False

    def _checkWin(self):
    # Check rows
        for row in self.grid:
            if np.all(row == self.currMove):
                return True

        # Check columns
        for col in self.grid.T:
            if np.all(col == self.currMove):
                return True

        # Check diagonals
        diag1 = np.diag(self.grid)
        diag2 = np.diag(np.fliplr(self.grid))
        if np.all(diag1 == self.currMove) or np.all(diag2 == self.currMove):
            return True

        return False



    def _changeMove(self):
        if self.currMove == 1:
            self.currMove = -1
        else:
            self.currMove = 1

Then comes the agent class which
- Uses TD(λ) algorithm to find the optimal policies for each state
- Stores the calculated optimal policies as a .npy file for later use
- Calculates the average return of the itself against a random player (makes random moves on its chance) periodically during training and plot it (for example if total training iterations is 10000, then calculate average return after each 500 steps, also for average return you should calculate return atleast 5 times and then take average)
- You can make additional functions

You can store all the encountered states in a numpy array (which will have 3 dims) and then store corresponding values for that particulare state in another array (will have 1 dims) and then you can store all these arrays in a .npy file for future use, so that you don't have to train the model each time you want to play TicTacToe

In [47]:
# Both the constructors and train() function can have any arguments you need
class TicTacToeAgent:
    def __init__(self, tic_tac_toe, alpha=0.1, gamma=0.9, epsilon=0.1, lambda_=0.5):
        self.tic_tac_toe = tic_tac_toe
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.lambda_ = lambda_
        # self.state_values = {}
        self.state_values = np.load('stored_data.npy', allow_pickle=True).item()
        self.eligibility_traces = {}

    def train(self, episodes=100000, evaluate_every=500):
        returns = []
        for episode in range(episodes):
            self.eligibility_traces.clear()
            state = self.tic_tac_toe.reset()[0]
            done = False
            total_return = 0
            while not done:
                action = self.choose_action(state)
                if action == None:
                    done = True
                    break
                next_state, reward, done = self.tic_tac_toe.act(action)
                total_return += reward

                td_target = reward if done else reward + self.gamma * self.get_state_value(next_state)
                td_error = td_target - self.get_state_value(state)

                self.update_state_value(state, td_error)
                state = next_state

            returns.append(total_return)

            if episode % evaluate_every == 0:
                avg_return = sum(returns[-5:]) / 5
                print(f"Episode: {episode}, Average Return: {avg_return}")

        return returns

    def choose_action(self, state):
        valid_actions = [i for i, val in enumerate(state.flatten()) if val == 0]
        if not valid_actions:
            return None  # No valid actions available, return None or handle as needed
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        else:
            return max(valid_actions, key=lambda a: self.get_state_value(state))


    def get_state_value(self, state):
        state_key = tuple(state.flatten())
        return self.state_values.get(state_key, 0)

    def update_state_value(self, state, td_error):
        state_key = tuple(state.flatten())
        self.state_values[state_key] = self.get_state_value(state) + self.alpha * td_error

    def load_data(self, file_path):
        loaded_data = np.load(file_path, allow_pickle=True)
        self.state_values = loaded_data['state_values']

    def save_data(self, file_path):
        np.save(file_path, {'state_values': self.state_values})


Now for evaluation purposes and for your self checking the code block below after running should:
- Initialize the agent and call the train function which trains the agent
- Load the stored state value data
- Start a single player game of TicTacToe which takes input from the user for moves according to the convention given below, where the trained Q values play as computer

In [48]:
'''
You will be asked to enter number corresponding to the boards where you want to make your move, for example in 1 3x3 TicTacToe:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 9
The model should train a 3x3 TicTacToe by default, you can definitely modify the values(of N, number of iterations etc) for your convenience but training model for bigger N might take lot of time
'''

tic_tac_toe = TicTacToe(n = 3)
agent = TicTacToeAgent(tic_tac_toe=tic_tac_toe)
returns = agent.train()
agent.save_data('stored_data.npy')

class TicTacToeGame:
    def __init__(self, agent):
        self.tic_tac_toe = TicTacToe()  # Initialize a TicTacToe game instance
        self.agent = agent

    def print_board(self):
        board = self.tic_tac_toe.grid
        for row in board:
            print(" | ".join(["X" if cell == 1 else "O" if cell == -1 else " " for cell in row]))
            print("-" * 9)

    def play_game(self):
        print("Welcome to Tic-Tac-Toe!")
        current_player = -1
        while True:
            self.print_board()
            if current_player == 1:
                action = int(input("Enter your move (1-9): ")) - 1
            else:
                action = self.agent.choose_action(self.tic_tac_toe.grid)

            state, reward, done = self.tic_tac_toe.act(action)
            if done:
                self.print_board()
                if reward == 1:
                    print("Congratulations! You win!")
                elif reward == -1:
                    print("Sorry, you lost!")
                else:
                    print("It's a draw!")
                break

            current_player = -current_player  # Switch between players


# Assuming 'agent' is already trained and initialized
agent = TicTacToeAgent(tic_tac_toe=TicTacToe())
game = TicTacToeGame(agent)
game.play_game()



Episode: 0, Average Return: 0.0
Episode: 500, Average Return: 0.6
Episode: 1000, Average Return: 0.6
Episode: 1500, Average Return: 0.6
Episode: 2000, Average Return: 1.0
Episode: 2500, Average Return: 0.8
Episode: 3000, Average Return: 1.0
Episode: 3500, Average Return: 0.0
Episode: 4000, Average Return: 0.6
Episode: 4500, Average Return: 0.8
Episode: 5000, Average Return: 1.0
Episode: 5500, Average Return: 0.6
Episode: 6000, Average Return: 1.0
Episode: 6500, Average Return: 0.2
Episode: 7000, Average Return: 1.0
Episode: 7500, Average Return: 0.6
Episode: 8000, Average Return: 1.0
Episode: 8500, Average Return: 0.0
Episode: 9000, Average Return: 0.6
Episode: 9500, Average Return: 0.2
Episode: 10000, Average Return: 1.0
Episode: 10500, Average Return: 0.2
Episode: 11000, Average Return: 0.0
Episode: 11500, Average Return: 0.8
Episode: 12000, Average Return: 1.0
Episode: 12500, Average Return: -0.2
Episode: 13000, Average Return: 1.0
Episode: 13500, Average Return: 0.4
Episode: 14000,

ValueError: invalid literal for int() with base 10: ''